In [2]:
# Version du projet pour l'expérimentation des bi-grammes

import string
import re
import operator
import math
import nltk
nltk.download()
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer

mystemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [20]:
#prend un tuple (mot,POS), et converti le POS en annotations pour le lemmatiseur
def get_wordnet_pos(word):
    tag = word[1][0]
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def lemmatisation(ligne):
    #Tableau de tous les tags non-désirés
    rejected = ['CC','IN','DT','EX','POS','PRP','PRP$','MD','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB','UH','TO','JJ','JJR','RB','RBR',',','','.',':','(',')','``',"''"]
    # supprime les espaces et sauts de lignes en début et fin de chaîne
    ligne.lstrip
    ligne.rstrip("\n")
    #On tokenize le texte pour appliquer des Part-Of-Speech
    tutu = nltk.pos_tag(nltk.word_tokenize(ligne))#liste de tuple
    #print(type(tutu[0]))
    
   
    i=0
    while i <len(tutu):
        mot = tutu[i]
        if mot[1] in rejected :
            tutu.remove(mot)
        else :
            g = re.sub('[\.]','',mot[0])
            motAPlacer=lemmatizer.lemmatize(g.lower(), get_wordnet_pos(mot))
            tutu[i] = motAPlacer
            i+=1
            
            
    return tutu   #!! On retourne une liste !!
    
def lemmatisationPourBigrammes(ligne):
    #Tableau de tous les tags non-désirés
    rejected = ['CC','IN','DT','EX','POS','PRP','PRP$','MD','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB','UH','TO',',','','.',':','(',')','``',"''"]
    # supprime les espaces et sauts de lignes en début et fin de chaîne
    ligne.lstrip
    ligne.rstrip("\n")
    #On tokenize le texte pour appliquer des Part-Of-Speech
    tutu = nltk.pos_tag(nltk.word_tokenize(ligne))#liste de tuple
    #print(type(tutu[0]))
    
   
    i=0
    while i <len(tutu):
        mot = tutu[i]
        if mot[1] in rejected :
            tutu.remove(mot)
        else :
            g = re.sub('[\.]','',mot[0])
            motAPlacer=lemmatizer.lemmatize(g.lower(), get_wordnet_pos(mot))
            tutu[i] = motAPlacer
            i+=1
            
            
    return tutu   #!! On retourne une liste !!    

print(lemmatisation("Editions of the Dewey Decimal Classifications The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad."))
#print(lemmatisation("Editions of the Dewey Decimal Classifications..\nThe present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad."))

['edition', 'dewey', 'decimal', 'classification', 'study', 'history', 'dewey', 'decimal', 'classification', 'edition', 'ddc', '1876', 'edition', '1971', 'edition', 'spite', 'ddc', 'life', 'story', 'biography', 'dewey', 'system', 'attempt', 'history', 'work', 'growth', 'librarianship', 'country']


In [4]:
def monstemmer(ligne) :
    ligne.lstrip
    ligne.rstrip("\n")
    tab = ligne.split()
    results =[]
    for mot in tab :
        g = re.sub('\.','',mot)
        results.append(mystemmer.stem(g.lower()))
    return results

print(monstemmer("You're my friend, friendy Terry"))
    

["you'r", 'my', 'friend,', 'friendi', 'terri']


In [42]:


# table renvoyant une liste de textes (sous forme d'ints représentant le numéro des textes) pour chaque mot
tableMotsGenerale = {}
# pour chaque numéro de texte renvoie le hashSet représentant les valeurs TFIDF de chaque mot
tableTextes = {}
# table renvoyant une liste de textes (sous forme d'ints représentant le numéro des textes) pour chaque bigramme
tableBiGenerale = {}
# pour chaque numéro de texte renvoie le hashSet représentant les valeurs TFIDF de chaque bigramme
tableTextesBi = {}

def acquisition(path):
    
    f=open(path,'r')
    
    # masque pour détecter les débuts de textes
    regexp=re.compile("^\.I\s*([0-9]*)\s*")
    
    # numéro du texte courant, valeur abérante par défaut
    numero = -1
    
    # tableau contenant chaque ligne du fichier
    listeLignes = f.readlines()
    
    # le nombre de mots rencontrés
    nombreMots = 0
    nombreBigrammes = 0
    
    for ligne in listeLignes:
        
        
        # on cherche la marque du début d'un nouveau ou du premier texte
        result = regexp.search(ligne)
        if result != None:
            # teste si on a déjà rencontré un texte avant
            if numero != -1:
                for mot,valeur in tableTextes[numero].items():
                    # TF, sans IDF car il manque le nombre de textes dans lesquels le mot apparait
                    tableTextes[numero][mot]=(valeur/nombreMots)
                for bigramme,valeur in tableTextesBi[numero].items():
                    tableTextesBi[numero][bigramme]=(valeur/nombreBigrammes)
            # on capture l'index du prochain texte 
            numero = int(result.group(1))
            # on initialise sa table de mots
            tableTextes[numero] = {}
            tableTextesBi[numero] = {}
             # on initialise le nombre de mots de celui-ci
            nombreMots = 0
            nombreBigrammes = 0
            print(numero)
            
        else:
            postTags = lemmatisation(ligne)
            postTagsBi = lemmatisationPourBigrammes(ligne)
            bigrammes = list(nltk.bigrams(postTagsBi))
            
            for bigramme in bigrammes:
                nombreBigrammes+=1
                # si le mot existe déjà dans la table on augmente son nombre d'apparitions
                if bigramme in tableTextesBi[numero]:
                    tableTextesBi[numero][bigramme]+=1
                # dans le cas contraire on ajoute le mot dans la table
                else:
                    tableTextesBi[numero][bigramme]=1
                    # si le mot a déjà été vu pour un autre texte, on ajoute ce texte dans la table générale
                    if bigramme in tableBiGenerale:
                        tableBiGenerale[bigramme].append(numero)
                    # sinon on ajoute le mot dans la table générale et on y associe ce texte
                    else:
                        tableBiGenerale[bigramme]=[numero]
            
            for mot in postTags:
                nombreMots+=1
                # si le mot existe déjà dans la table on augmente son nombre d'apparitions
                if mot in tableTextes[numero]:
                    tableTextes[numero][mot]+=1
                # dans le cas contraire on ajoute le mot dans la table
                else:
                    tableTextes[numero][mot]=1
                    # si le mot a déjà été vu pour un autre texte, on ajoute ce texte dans la table générale
                    if mot in tableMotsGenerale:
                        tableMotsGenerale[mot].append(numero)
                    # sinon on ajoute le mot dans la table générale et on y associe ce texte
                    else:
                        tableMotsGenerale[mot]=[numero]

    # on a parcouru tous les termes, on peut appliquer l'IDF du TFIDF au TF des bigrammes
    for texte in tableTextesBi.keys():
        listeASupprimer = []
        for bigramme in tableTextesBi[texte].keys():
            tableTextesBi[texte][bigramme]/=len(tableTextesBi[texte])
            #if tableTextesBi[texte][bigramme] < 0.0008 :#0.0003 ?
             #   listeASupprimer.append(bigramme)
        for bigramme in listeASupprimer:
            del(tableTextesBi[texte][bigramme])          
         
    # mise à jour de tableMotsGeneraleBi
    tableBiGenerale.clear()
    for texte,scores in tableTextesBi.items():
        for bigramme,score in scores.items():
            if not(bigramme in tableBiGenerale):
                tableBiGenerale[bigramme]=[texte]
            else:
                tableBiGenerale[bigramme].append(texte)
        
    # on a parcouru tous les termes, on peut appliquer l'IDF du TFIDF au TF des 1-grammes
    for texte in tableTextes.keys():
        listeASupprimer = []
        for mot in tableTextes[texte].keys():
            tableTextes[texte][mot]*=(math.log(len(tableTextes)/len(tableMotsGenerale[mot]))**2)/len(tableTextes[texte])
            if tableTextes[texte][mot] < 0.0008 :#0.0003 ?
                listeASupprimer.append(mot)
        for mot in listeASupprimer:
            del(tableTextes[texte][mot])
    
    # mise à jour de tableMotsGenerale
    tableMotsGenerale.clear()
    for texte,scores in tableTextes.items():
        for mot,score in scores.items():
            if not(mot in tableMotsGenerale):
                tableMotsGenerale[mot]=[texte]
            else:
                tableMotsGenerale[mot].append(texte)
            
    
    for mot,score in tableTextesBi[1].items():
        print(mot," : ",len(tableTextesBi)," / ",len(tableBiGenerale[mot]), " = ",math.log(len(tableTextesBi)/len(tableBiGenerale[mot])))
    
    
    f.close()
    
    return

# textes : acquisition met à jour la base de données des corpus
acquisition("./CISI.ALLnettoye")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [35]:
print(len(tableMotsGenerale))

6094


In [43]:
def afficherTableTexte(texte):
    for mot,score in tableTextes[texte].items():
            print(mot," : ",len(tableTextes)," / ",len(tableMotsGenerale[mot]), " ; ",tableTextes[texte][mot])
def afficherTableTexteBi(texte):
    for mot,score in tableTextesBi[texte].items():
            print(mot," : ",len(tableTextesBi)," / ",len(tableBiGenerale[mot]), " ; ",tableTextesBi[texte][mot])
    
afficherTableTexteBi(2)

('technology', 'rather')  :  1460  /  1  ;  0.00028264556246466934
('library', 'user')  :  1460  /  15  ;  0.00028264556246466934
('other', 'way')  :  1460  /  2  ;  0.00028264556246466934
('information', 'use')  :  1460  /  5  ;  0.00028264556246466934
('library', 'act')  :  1460  /  1  ;  0.00028264556246466934
('use', 'made')  :  1460  /  1  ;  0.00028264556246466934
('only', 'one')  :  1460  /  14  ;  0.00028264556246466934
('act', 'use')  :  1460  /  1  ;  0.00028264556246466934
('use', 'information')  :  1460  /  12  ;  0.00028264556246466934
('person', 'library')  :  1460  /  3  ;  0.00028264556246466934
('transfer', 'library')  :  1460  /  1  ;  0.00028264556246466934
('technical', 'library')  :  1460  /  7  ;  0.0005652911249293387
('contact', 'colleague')  :  1460  /  1  ;  0.00028264556246466934
('user', 'technical')  :  1460  /  1  ;  0.00028264556246466934
('act', 'channel')  :  1460  /  1  ;  0.00028264556246466934
('6300', 'act')  :  1460  /  1  ;  0.00028264556246466934

In [68]:
monFichier = open("Dict.txt",'w')
for i in range(200,221) :
    monFichier.write("Texte "+str(i)+"\n")
    for mots,score in tableTextes[i].items() :
        monFichier.write(mots+";"+str(score)+"\n")
monFichier.close()

mean = 0
sigma = 0
for texte in tableTextesBi.keys():
    mean += len(tableTextesBi[texte])
mean /= len(tableTextesBi)
for texte in tableTextesBi.keys():
    sigma += (mean-len(tableTextesBi[texte]))**2
sigma /= len(tableTextesBi)
               
print(mean,math.sqrt(sigma))

45.64383561643836 22.749571643122913


In [54]:
def traitementRequete(Requete):
    # Donne, pour chaque texte, un score de pertinance par rapport à la requête
    scores = {}
    normes = {}
    tfRequete = {}
    
     
    mots = lemmatisation(Requete)
    
    '''
    # Calcul des Term Frequency de la requête
    for mot in mots:
        if mot in tfRequete:
            tfRequete[mot]+=1
        else:
            tfRequete[mot]=1
    
    # Division des TF par le nombre de mots de la requête et calcul de la norme de tfRequete
    normeTfRequete = 0  
    for mot in tfRequete:
        tfRequete[mot]/=len(mots)
        normeTfRequete+=tfRequete[mot]**2
    normeTfRequete=math.sqrt(normeTfRequete)    
    '''

    # Calcul des scores
    for mot in mots:
        if mot in tableMotsGenerale:
            listeTextes = tableMotsGenerale[mot]
            for texte in listeTextes:
                if texte in scores:
                    scores[texte]+=tableTextes[texte][mot]#*tfRequete[mot]
                    normes[texte]+=(tableTextes[texte][mot])**2
                else:
                    scores[texte]=tableTextes[texte][mot]#*tfRequete[mot]
                    normes[texte]=(tableTextes[texte][mot])**2
    
    
        
    
        
    for texte in scores.keys():
        normes[texte]=math.sqrt(normes[texte])
        scores[texte]=scores[texte]/(normes[texte]*math.sqrt(len(mots)))#*normeTfRequete
    
    
    # enlève les textes avec un petit score (inférieur à la moitié du meilleur)
    listeASupprimer = []
    for texte,score in scores.items():
        if score<(0.5): #entre 0.5 et 0.4
            listeASupprimer.append(texte)
    for texte in listeASupprimer:
        del scores[texte]
    
    return scores


def printScores(scores):
    print((sorted(scores.items(), key=operator.itemgetter(1)))[::-1])

scores = traitementRequete("Dewey Number Classification")
len(scores)



278

In [47]:
def acquisitionDesRequetes(path):
    f=open(path,'r')
    
    # si true, on capture une requête
    catch = False
    
    # Table contenant pour chaque numéro de requête un texte de requête
    tableRequete={}
    
    # masque pour détecter les numéros de requêtes
    regexNumero=re.compile("^\.I\s*([0-9]*)\s*")
    
    # masque pour détecter les débuts de requêtes
    regexContenu=re.compile("^\.W")
    
    # masque pour détecter les fins de requêtes
    regexFinCatch=re.compile("^\.[BI]")
    
    # numéro de la requête courante
    numero = 1
    
    # tableau contenant chaque ligne du fichier
    listeLignes = f.readlines()
    
    for ligne in listeLignes:
        # supprime les espaces et sauts de lignes en début et fin de chaîne
        ligne.lstrip
        ligne.rstrip("\n")
        
        # fin de texte
        result = regexFinCatch.search(ligne)
        if result != None:
            catch = False
        
        if catch:
            tableRequete[numero]+=ligne
        
        # numero de requête
        result = regexNumero.search(ligne)
        if result != None:
            # on capture son index 
            numero = int(result.group(1))
            # on initialise son contenu
            tableRequete[numero] = ""
        # debut de texte
        result = regexContenu.search(ligne)
        if result != None:
            catch = True
            
    f.close()
    
    return tableRequete



In [55]:
def evaluationListeRequetes(listeRequetes):
    listeScoresRequetes = {}
    for numero,requete in listeRequetes.items():
        listeScoresRequetes[numero] = traitementRequete(requete)
    #print(listeScoresRequetes[8])    
    return listeScoresRequetes
        

def ecrireListeScoresRequetes(path,listeScoresRequetes):
    f=open(path,'w')
    for numeroRequete,scores in listeScoresRequetes.items():
        for numeroTexte,score in scores.items():
            f.write(str(numeroRequete)+" "+str(numeroTexte)+" "+str(score)+'\n')
    
    
    f.close()
    return

# textes : acquisition met à jour la base de données des corpus
#acquisition("./CISI.ALLnettoye")
# requêtes : ecrireListeScoresRequetes écrit dans un fichier le score de pertinence pour chaque texte obtenu 
# pour chaque requête (evaluationlisteRequetes apelle traitementRequete sur chaque requête venant de
# acquisitionDesRequetes)
ecrireListeScoresRequetes("./resultats",evaluationListeRequetes(acquisitionDesRequetes("./CISI_dev.QRY")))

print("Fini !")
# Evaluer les résultats avec le script perl : ./eval.pl CISI_dev.REL resultats 

Fini !
